In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import time
import datetime
from pymongo import MongoClient
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), "..", 'api', 'config')))

from config import Config

In [6]:
client = MongoClient(Config.MONGO_URI)
db = client["nugenesis-bitcoin"]
blockchain_collection = db["cryptocompare_v2"]


In [9]:
# Retrieve all documents from the collection
cursor = blockchain_collection.find()

blockchain_df = pd.DataFrame(list(cursor))

blockchain_df.head()


,_id,id,symbol,time,zero_balance_addresses_all_time,unique_addresses_all_time,new_addresses,active_addresses,transaction_count,transaction_count_all_time,large_transaction_count,average_transaction_value,block_height,hashrate,difficulty,block_time,block_size,current_supply
0,669515d0aecca53f69ee8653,1182,BTC,2019-01-23 02:00:00,454954548,477751652,353870,708846,293494,375977161,5876,4.013580,559814,4.303635e+07,5.883988e+12,587.213793,1026111,1.749766e+07
1,669515d0aecca53f69ee8654,1182,BTC,2019-01-24 02:00:00,455303669,478106357,354705,660872,308146,376285307,8733,4.807922,559976,4.744141e+07,5.883988e+12,532.689441,911348,1.749968e+07
2,669515d0aecca53f69ee8655,1182,BTC,2019-01-25 02:00:00,455627420,478464509,358152,657356,301085,376586392,9017,6.076248,560114,4.009172e+07,5.883988e+12,630.343066,1017360,1.750141e+07
3,669515d0aecca53f69ee8656,1182,BTC,2019-01-26 02:00:00,455956146,478813652,349143,621149,288285,376874677,5667,3.318763,560264,4.410649e+07,5.883988e+12,572.966443,980735,1.750328e+07
4,669515d0aecca53f69ee8657,1182,BTC,2019-01-27 02:00:00,456233400,479109651,295999,532591,276651,377151328,4272,2.657709,560401,4.029981e+07,5.883988e+12,627.088235,881242,1.750500e+07


In [10]:
blockchain_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   _id                              2000 non-null   object 
 1   id                               2000 non-null   int64  
 2   symbol                           2000 non-null   object 
 3   time                             2000 non-null   object 
 4   zero_balance_addresses_all_time  2000 non-null   int64  
 5   unique_addresses_all_time        2000 non-null   int64  
 6   new_addresses                    2000 non-null   int64  
 7   active_addresses                 2000 non-null   int64  
 8   transaction_count                2000 non-null   int64  
 9   transaction_count_all_time       2000 non-null   int64  
 10  large_transaction_count          2000 non-null   int64  
 11  average_transaction_value        2000 non-null   float64
 12  block_height        

In [11]:
blockchain_df.describe()

,id,zero_balance_addresses_all_time,unique_addresses_all_time,new_addresses,active_addresses,transaction_count,transaction_count_all_time,large_transaction_count,average_transaction_value,block_height,hashrate,difficulty,block_time,block_size,current_supply
count,2000.0,2.000000e+03,2.000000e+03,2000.000000,2.000000e+03,2000.00000,2.000000e+03,2000.000000,2000.000000,2000.000000,2.000000e+03,2.000000e+03,2000.000000,2.000000e+03,2.000000e+03
mean,1182.0,8.646067e+08,9.028415e+08,427414.736500,8.964050e+05,332581.97500,6.828571e+08,15815.271500,6.522274,705478.434000,2.283584e+08,3.125266e+13,592.838643,1.317762e+06,1.879873e+07
std,0.0,2.441357e+08,2.526035e+08,78192.052582,1.484740e+05,105910.80677,1.779888e+08,5508.035409,6.474644,87169.495546,1.634908e+08,2.215151e+13,71.946228,2.970059e+05,6.068638e+05
min,1182.0,4.549545e+08,4.777517e+08,0.000000,0.000000e+00,0.00000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,1.749766e+07
25%,1182.0,6.428527e+08,6.730436e+08,382003.750000,7.934198e+05,268946.50000,5.359389e+08,12003.000000,2.475527,633056.250000,1.120018e+08,1.554675e+13,548.689794,1.116301e+06,1.839635e+07
50%,1182.0,8.654255e+08,9.069650e+08,421026.500000,9.101485e+05,306358.00000,6.789688e+08,15308.500000,3.975564,705390.000000,1.671125e+08,2.267415e+13,584.470474,1.273912e+06,1.884810e+07
75%,1182.0,1.073540e+09,1.117156e+09,470441.500000,9.881572e+05,355260.25000,8.104269e+08,19041.000000,8.486301,779077.000000,3.135640e+08,4.305384e+13,625.925815,1.533356e+06,1.930698e+07
max,1182.0,1.276486e+09,1.329284e+09,907493.000000,1.368446e+06,927010.00000,1.041539e+09,34806.000000,79.430165,852221.000000,7.219917e+08,8.810419e+13,1380.052632,2.517537e+06,1.972033e+07


In [13]:
print(blockchain_df.isna().sum())

_id                                0
id                                 0
symbol                             0
time                               0
zero_balance_addresses_all_time    0
unique_addresses_all_time          0
new_addresses                      0
active_addresses                   0
transaction_count                  0
transaction_count_all_time         0
large_transaction_count            0
average_transaction_value          0
block_height                       0
hashrate                           0
difficulty                         0
block_time                         0
block_size                         0
current_supply                     0
dtype: int64
